In [47]:
import pandas as pd
import numpy as np
import scipy as sc
from IPython.display import display
#import folium
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
#from sklearn.pipeline import make_pipeline
#from imblearn.under_sampling import RandomUnderSampler 
#from imblearn.over_sampling import SMOTE,RandomOverSampler
#from sklearn.metrics import recall_score,precision_score

In [14]:
amz = pd.read_csv("amazon_labelled_locations.csv",thousands = ",")
moodys = pd.read_csv("MoodysData-ACS.csv",thousands=",")
us_county = pd.read_csv("national_county.txt", names = ["state","state_code","county_code","county","FIPS_class_code"],dtype=str)
#print amz

In [15]:
us_state_abbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO",
                   "Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI","Idaho": "ID",
                   "Illinois": "IL","Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA",
                   "Maine": "ME","Maryland": "MD","Massachusetts": "MA","Michigan": "MI","Minnesota": "MN",
                   "Mississippi": "MS","Missouri": "MO","Montana": "MT","Nebraska": "NE","Nevada": "NV",
                   "New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM","New York": "NY","North Carolina": "NC",
                   "North Dakota": "ND","Ohio": "OH","Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA",
                   "Rhode Island": "RI","South Carolina": "SC","South Dakota": "SD","Tennessee": "TN","Texas": "TX",
                   "Utah": "UT","Vermont": "VT","Virginia": "VA","Washington": "WA","West Virginia": "WV",
                   "Wisconsin": "WI","Wyoming": "WY"}

In [16]:
display(amz.head(1))
display(moodys.head(1))
display(us_county.head(1))

,State,Code,Street Address,Square Feet,Year Opened,Description of Operation,Type,County,City,Year
0,Arizona,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,May-15,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Maricopa County,Phoenix,2015


,﻿,Unnamed: 1,Code,Include in Amazon Dataset,Years,Current Senior Most Rating*,Tax Backed Rating Description,State,Population (ACS Data),Per Capita Income (ACS Data),Median Family Income (ACS Data),Median Home Value (ACS Data),Median Gross Rent (ACS Data),Occupied Housing Units (ACS Data),Seasonal Homes (ACS Data),No. Persons/Household (ACS Data),Median Age (ACS Data),% Below Poverty Level (ACS Data)
0,Median,Median,NaN,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,state,state_code,county_code,county,FIPS_class_code
0,AL,01,001,Autauga County,H1


In [26]:
us_county["geoid"]="050"+"0000"+"US"+us_county["state_code"]+us_county["county_code"]
us_county["county_state"]=us_county["county"]+", "+us_county["state"]
us_county_trimmed = us_county.drop(["state","state_code","county_code","county","FIPS_class_code"],axis=1)


moodys_2012 = moodys[moodys["Years"]==2012]
moodys_2012 = moodys_2012.rename(columns={"Unnamed: 1":"county_state"})
#moodys_2012 = moodys_2012.drop(["Unnamed: 0"],axis=1)


amz["county_state"]=amz.apply(axis=1,func=lambda row:str(row["County"])+", " +us_state_abbrev[row["State"]])
amz_trimmed = amz[["county_state"]]
#t1 = pd.DataFrame(np.ones((amz_trimmed.shape[0],1)),columns=["warehouse"])
amz_trimmed=amz_trimmed.assign(warehouse=1)
#amz_trimmed["warehouse"]=t1["warehouse"]
t1 = pd.merge(how="left",left=moodys_2012,right=amz_trimmed,left_on="county_state",right_on="county_state")


t1["warehouse"].fillna(0,inplace=True)


t2 = pd.merge(how="left",left =t1,left_on="county_state",right=us_county_trimmed,right_on="county_state")
#don't remove dupes add them to a new count column

final= t2[t2["geoid"].notnull()]


In [28]:
final.to_csv('data_organized1.csv')


In [49]:
#Reupload file with three new data points and cleaned
cleandata = pd.read_csv("data_organized1.csv",thousands = ",")
df_x = cleandata.iloc[:,1:14]
df_y = cleandata['warehouse']

In [60]:
#display data
#display(df_y)

In [58]:
#train a ridge regression model
#from sklean.linear_model import Ridge
def ridge_regression(alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha, normalize = True)
    ridgereg.fit(df_x,df_y)
    y_pred = ridgereg.predict(df_x)
    
    #Return the results
    rss = sum((y_pred-df_y)**2)
    ret = [rss]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)
    return ret

predictors = df_x

#set the different values of alpha to be tested
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]

#Initialize the dataframe for storing coefficients
col = ['rss', 'intercept'] + ['coef_x_%d'%i for i in range(1,14)]
ind = ['alpha_%.2g'%alpha_ridge[i] for i in range(0,10)]
coef_matrix_ridge = pd.DataFrame(index = ind, columns = col)

#models_to_plot =  {1e-15:231, 1e-10:232, 1e-4:233, 1e-3:234, 1e-2:235, 5:236}

for i in range(10):
    coef_matrix_ridge.iloc[i,] = ridge_regression(alpha_ridge[i])

In [59]:
display(coef_matrix_ridge)

,rss,intercept,coef_x_1,coef_x_2,coef_x_3,coef_x_4,coef_x_5,coef_x_6,coef_x_7,coef_x_8,coef_x_9,coef_x_10,coef_x_11,coef_x_12,coef_x_13
alpha_1e-15,138.447,-0.09241556,0.006420337,-1.377857e-09,6.569491e-08,-8.612048e-07,1.2091e-06,1.96471e-06,-2.546995e-07,0.0003048431,3.502021e-06,3.746695e-07,0.004206902,-0.005476137,-0.001428092
alpha_1e-10,138.447,-0.09241556,0.006420337,-1.377857e-09,6.569491e-08,-8.612047e-07,1.2091e-06,1.96471e-06,-2.546995e-07,0.0003048431,3.502021e-06,3.746696e-07,0.004206901,-0.005476138,-0.001428092
alpha_1e-08,138.447,-0.09241575,0.006420341,-1.377861e-09,6.569502e-08,-8.611991e-07,1.209139e-06,1.964705e-06,-2.546999e-07,0.000304843,3.502006e-06,3.746717e-07,0.004206852,-0.005476146,-0.001428084
alpha_0.0001,138.4636,-0.09417554,0.006458714,-1.411798e-09,6.671182e-08,-8.074812e-07,1.576527e-06,1.925656e-06,-2.582978e-07,0.000303811,3.35269e-06,3.955026e-07,0.00373204,-0.005558914,-0.001353345
alpha_0.001,139.2224,-0.1040694,0.006688039,-1.587179e-09,7.173521e-08,-4.959153e-07,3.733961e-06,1.693703e-06,-2.809998e-07,0.0002980556,2.479938e-06,5.252795e-07,0.001014667,-0.006052849,-0.0009200539
alpha_0.01,143.1889,-0.1135248,0.007130646,-1.470756e-09,6.44614e-08,-6.660879e-09,7.450016e-06,1.275858e-06,-3.410095e-07,0.0002901539,9.865119e-07,9.054765e-07,-0.0024486,-0.007045824,-0.0002405431
alpha_1,159.7671,-0.0655716,0.002613256,3.0268e-10,8.128154e-08,4.768641e-08,3.108869e-06,1.177611e-06,1.163609e-07,0.0001311137,1.574355e-07,2.128289e-06,0.01505072,-0.004176202,-0.0005433849
alpha_5,188.298,-0.02648898,0.0008992206,2.074948e-10,7.931367e-08,2.529231e-08,1.689769e-06,6.805313e-07,1.220798e-07,7.244574e-05,7.816063e-08,1.388664e-06,0.009868564,-0.001700022,-0.000465914
alpha_10,209.7971,0.008843082,0.0004987981,1.382322e-10,5.761848e-08,1.628212e-08,1.140968e-06,4.598914e-07,8.721616e-08,4.792284e-05,4.985405e-08,9.30601e-07,0.00644409,-0.001007867,-0.0003494475
alpha_20,230.5575,0.0408031,0.0002612989,8.25233e-11,3.6237e-08,9.548356e-09,6.941097e-07,2.795384e-07,5.424657e-08,2.867003e-05,2.909926e-08,5.581376e-07,0.003789944,-0.0005602747,-0.0002263047
